In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# %load functions/houseFunctions.py
'''
Parse 1976-2016 house data from https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IG0UN2

@relFilePath : file path of house results data, relative to python notebook

@return:  dataframe indexed by (year, state, district)
'''

def parse_data(relFilePath):

    data_df = pd.read_csv(relFilePath)

    winners_df = pd.DataFrame()
    for key, shard in data_df.groupby(['year', 'state_po', 'district']):
    	winners_df = winners_df.append(shard.loc[shard['candidatevotes'].idxmax()])
    return winners_df
    

In [3]:
full_interest = parse_data("Datasets/fec/1976-2016-house.csv")

In [4]:
full_interest

,candidate,candidatevotes,district,office,party,special,stage,state,state_cen,state_fips,state_ic,state_po,totalvotes,version,writein,year
9,Don Young,83722.0,0.0,US House,republican,0.0,gen,Alaska,94.0,2.0,81.0,AK,118208.0,20171005.0,0.0,1976.0
198,Jack Edwards,98257.0,1.0,US House,republican,0.0,gen,Alabama,63.0,1.0,41.0,AL,157170.0,20171005.0,0.0,1976.0
288,"William L. ""Bill"" Dickinson",90069.0,2.0,US House,republican,0.0,gen,Alabama,63.0,1.0,41.0,AL,156362.0,20171005.0,0.0,1976.0
185,Bill Nichols,106935.0,3.0,US House,democrat,0.0,gen,Alabama,63.0,1.0,41.0,AL,108048.0,20171005.0,0.0,1976.0
479,Tom Bevill,141490.0,4.0,US House,democrat,0.0,gen,Alabama,63.0,1.0,41.0,AL,176022.0,20171005.0,0.0,1976.0
272,Ronnie G. Flippo,113553.0,5.0,US House,democrat,0.0,gen,Alabama,63.0,1.0,41.0,AL,113560.0,20171005.0,0.0,1976.0
418,"John H. Buchanan, Jr.",92113.0,6.0,US House,republican,0.0,gen,Alabama,63.0,1.0,41.0,AL,162518.0,20171005.0,0.0,1976.0
94,Walter Flowers,110496.0,7.0,US House,democrat,0.0,gen,Alabama,63.0,1.0,41.0,AL,110501.0,20171005.0,0.0,1976.0
662,Bill Alexander,116217.0,1.0,US House,democrat,0.0,gen,Arkansas,71.0,5.0,42.0,AR,168782.0,20171005.0,0.0,1976.0
524,Jim Guy Tucker,144780.0,2.0,US House,democrat,0.0,gen,Arkansas,71.0,5.0,42.0,AR,167607.0,20171005.0,0.0,1976.0


In [35]:
# Make a dummy dataframe so everyone else can make complete dataframes
dummy_df = full_interest[['district','state_po','year']].reset_index().drop('index', axis=1)
dummy_df = dummy_df.rename(columns={'state_po' : 'state'})
dummy_df = dummy_df.astype({'year' : int, 'district' : int})
pickle.dump(dummy_df, open('Datasets/dummy_df.p', 'wb'))

# Load the file
#pickle.load(open('Datasets/dummy_df.p', 'rb'))

In [42]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
